In [15]:
import pandas as pd
import numpy as np

orders = pd.read_csv("datasets/orders/orders.csv")
departments = pd.read_csv("datasets/departments/departments.csv")
aisles = pd.read_csv("datasets/aisles/aisles.csv")
order_products_train = pd.read_csv("datasets/order_products__train/order_products__train.csv")
order_products_prior = pd.read_csv("datasets/order_products__prior/order_products__prior.csv")
products = pd.read_csv("datasets/products/products.csv")

In [16]:
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
%matplotlib inline

In [17]:
def extract_cust_data(orders,products,aisles,departments, order_products_prior, user_id):
    
    # Extracting orders of a particular user_id
    user_data = orders[orders.user_id==user_id]
    
    cust_orders = pd.DataFrame();
    
    # Getting all information from order_products_prior dataframe wrt user_id
    for order in (user_data[user_data.eval_set=='prior'].order_id):
        cust_orders = cust_orders.append(order_products_prior[order_products_prior.order_id==order])
    
    
    # Assuming '0': 'saturday', '1' : 'sunday', '2': 'monday', '3':'tuesday',  '4':'wednesday', '5':'thursday', '6':'friday'
    # and creating a dataframe for reference
    dow = pd.DataFrame({'num_dow' : pd.Series([0,1,2,3,4,5,6], index=[0,1,2,3,4,5,6]),
     'day' : pd.Series(['Saturday','Sunday','Monday','Tuesday','Wednesday','Thursday','Friday'], index=[0,1,2,3,4,5,6])})
    
    # Merging all important data from all dataframes
    cust_data = cust_orders.merge(user_data[user_data.eval_set=='prior'], left_on='order_id', right_on='order_id', how='outer')
    cust_data = cust_data.merge(products , left_on = 'product_id', right_on = 'product_id' , how= 'left')
    cust_data = cust_data.merge(aisles , left_on = 'aisle_id', right_on = 'aisle_id' , how= 'left')
    cust_data = cust_data.merge(departments , left_on = 'department_id', right_on = 'department_id' , how= 'left')
    cust_data = cust_data.merge(dow , left_on = 'order_dow', right_on = 'num_dow' , how= 'left')
    cust_data = cust_data[['user_id','order_id','order_number','product_id','aisle_id','department_id','eval_set','add_to_cart_order','reordered','order_dow','order_hour_of_day','days_since_prior_order','product_name','aisle','department','day']]
    
    return cust_data

In [18]:
cust1=extract_cust_data(orders,products,aisles,departments, order_products_prior, 1)
display(cust1[0:5])
print(len(orders))

,user_id,order_id,order_number,product_id,aisle_id,department_id,eval_set,add_to_cart_order,reordered,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle,department,day
0,1,2539329,1,196,77,7,prior,1,0,2,8,NaN,Soda,soft drinks,beverages,Monday
1,1,2539329,1,14084,91,16,prior,2,0,2,8,NaN,Organic Unsweetened Vanilla Almond Milk,soy lactosefree,dairy eggs,Monday
2,1,2539329,1,12427,23,19,prior,3,0,2,8,NaN,Original Beef Jerky,popcorn jerky,snacks,Monday
3,1,2539329,1,26088,23,19,prior,4,0,2,8,NaN,Aged White Cheddar Popcorn,popcorn jerky,snacks,Monday
4,1,2539329,1,26405,54,17,prior,5,0,2,8,NaN,XL Pick-A-Size Paper Towel Rolls,paper goods,household,Monday


3421083


In [79]:
# data preparation

def average_prev_days(cust):
    prior_days=[]
    for ord_num in cust1.order_number.unique():
        prior_days.append(cust1[cust1.order_number==ord_num].days_since_prior_order.values[0])

    prior_days = [x for x in prior_days if not np.isnan(x)]
    average_prior_days = np.mean(prior_days)
    
    return average_prior_days

